<a href="https://colab.research.google.com/github/sumithkumar07/Fashion-MNIST---Image-Classification-CNN---End-to-End-Deep-Learning-Project/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

In [92]:
# set random seeds for reproducibility
torch.manual_seed(42)

(2365, 785)

In [107]:
train = pd.read_csv('/content/fashion-mnist_train.csv')
test = pd.read_csv('/content/fashion-mnist_test.csv')

In [95]:
# train test split
x = train.iloc[:, 1:].values
y =train.iloc[:, 0].values

In [108]:
x_test = test.iloc[:, 1:].values
y_test = test.iloc[:, 0].values

In [120]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [121]:
# scaling the data

x_train = x_train/255.0
x_test = x_test/255.0

In [122]:
# create customdata class
class CustomData(Dataset):

  def __init__(self, features, labels):

    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.int64)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [123]:
train_dataset = CustomData(x_train, y_train)
test_dataset = CustomData(x_test, y_test)

In [124]:
# create train and test data loader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [125]:
# define nn class

class MyNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )

  # The forward method should be at the same level as __init__
  def forward(self, x):
    return self.model(x)

In [126]:
# set learning rate and epochs

learning_rate = 0.1
epochs = 100

In [127]:
# instatiate the model

model = MyNN(x_train.shape[1])

# loss function

criterion = nn.CrossEntropyLoss()

# optimizer

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [128]:
# traning loop

for epoch in range (epochs):

  total_epoch_loss = 0

  for batch_features , batch_labels in train_loader:

    # forward pass

    outputs = model(batch_features)

    # loss calculate
    loss = criterion(outputs, batch_labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()

    # updates grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  print(f"Epoch {epoch+1}/{epochs}, Loss: {total_epoch_loss/len(train_loader)}")

Epoch 1/100, Loss: 1.3278635867098545
Epoch 2/100, Loss: 0.792542982820078
Epoch 3/100, Loss: 0.6729751239854393
Epoch 4/100, Loss: 0.596674716958763
Epoch 5/100, Loss: 0.5448368441128562
Epoch 6/100, Loss: 0.4997357842347301
Epoch 7/100, Loss: 0.47225829726415325
Epoch 8/100, Loss: 0.4502764385431371
Epoch 9/100, Loss: 0.4219666191026674
Epoch 10/100, Loss: 0.41598508787070604
Epoch 11/100, Loss: 0.38431967166087305
Epoch 12/100, Loss: 0.4052959034840266
Epoch 13/100, Loss: 0.357615368163332
Epoch 14/100, Loss: 0.33454560203121064
Epoch 15/100, Loss: 0.3365736832538395
Epoch 16/100, Loss: 0.3109263594374589
Epoch 17/100, Loss: 0.30686202777404314
Epoch 18/100, Loss: 0.2820321686618717
Epoch 19/100, Loss: 0.29030079864863806
Epoch 20/100, Loss: 0.28534092732989197
Epoch 21/100, Loss: 0.2612681715676548
Epoch 22/100, Loss: 0.28003334959453724
Epoch 23/100, Loss: 0.2539014662781083
Epoch 24/100, Loss: 0.24950450151524645
Epoch 25/100, Loss: 0.23599311461051306
Epoch 26/100, Loss: 0.22762

In [118]:
# set model to eva mode
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [119]:
# evaluation code

total = 0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)
    total += batch_labels.size(0)
    correct += (predicted == batch_labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy}%")

Accuracy: 85.08333333333333%


In [131]:
# check fo gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device:{device}")

Using device:cpu
